# Day 12

- part 1 with Neo4j and QPPs
- part 1 with a bad memoized recursive approach - part 2 does not scale
- part 1 and 2 with a neater fully recursive approach

## Neo4j and QPPs

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

In [1]:
input_file = "input.txt"

In [ ]:
file = open(input_file, 'r')
for ix, line in enumerate(file):
    row, groups =line.split(' ')
    groups = [int(g) for g in groups.split(',')]
    qpp = '\n'.join([f"MATCH (head:Head WHERE head.row_id={ix})",
    "(()-->(:OK|UNKNOWN))*",
    "(()-->(:OK|UNKNOWN))+\n".join([f"(()-->(g_{jx}:NOK|UNKNOWN)){{{g}}}" for jx, g in enumerate(groups)]),
    "(()-->(:OK|UNKNOWN|Tail))*(:Tail)",
    "WITH DISTINCT head, "+', '.join([f"g_{jx}" for jx, _ in enumerate(groups)]),
    "WITH head, count(*) as matches",
    "SET head.matches = matches"])
    gds.run_cypher("""
    CREATE (:Line {id:$ix, row:$row, groups:$groups, qpp:$qpp})
    """, {'ix':ix, 'row':row, 'groups':groups, 'qpp':qpp})

In [ ]:
for id, row in gds.run_cypher("""MATCH (line:Line) RETURN line.id AS id, line.row AS row""").itertuples(index=False):
    gds.run_cypher("""
    WITH $row AS row
    UNWIND split(row, '') AS c
    CREATE (s:Spring {row_id:$id, state:c})
    WITH collect(s) AS springs
    CALL apoc.nodes.link(springs, 'NEXT')
    """, {'row':row, 'id':id})


In [ ]:
queries = """MATCH (s:Spring)
WHERE NOT EXISTS {()-[:NEXT]->(s)}
SET s:First;
MATCH (s:Spring)
WHERE NOT EXISTS {(s)-[:NEXT]->()}
SET s:Last;
MATCH (s:Spring {state:'.'})
SET s:OK;
MATCH (s:Spring {state:'#'})
SET s:NOK;
MATCH (s:Spring {state:'?'})
SET s:UNKNOWN;
MATCH (f:First)
MERGE (h:Head {row_id:f.row_id})
MERGE (h)-[:NEXT]->(f);
MATCH (l:Last)
MERGE (t:Tail {row_id:l.row_id})
MERGE (l)-[:NEXT]->(t)
""".split(';')
for query in queries:
    gds.run_cypher(query)

In [ ]:
gds.run_cypher("""CALL apoc.periodic.iterate('MATCH (l:Line) RETURN l.qpp AS qpp',
'CALL apoc.cypher.runWrite(qpp,{}) YIELD value RETURN value.count AS count',
{parallel:TRUE})""")

In [ ]:
gds.run_cypher("""MATCH (h:Head)
RETURN sum(h.matches) AS part1""")

## Part 1 - Ugly dynamic programming

In [2]:
from functools import cache
from collections import deque

In [3]:
@cache
def arrangements(chunk, groups):
    #print(prefix,'call ', chunk, groups) 
    res = set()

    if len(chunk) == 0 and len(groups) > 0:
        return set()

    if len(groups) == 0:
        if '#' in chunk:
            return set()
        res_nosearch = set(['.'*len(chunk)])
        #print(prefix,'no search', res_nosearch)
        return res_nosearch

    size, *others = groups
    others = tuple(others)
    fifo = chunk

    skipped = 0
    while len(fifo) > 0:
        candidate = fifo[0:size]
        if len(candidate) < size:
            break
        if len(fifo) > size:
            next_char = fifo[size]
        else:
            next_char = ''
        if not '.' in candidate and next_char != '#':
            #print(prefix,'skipped',skipped,'size',size,'next c',next_char)
            start = "."*skipped + "#"*size + ('.' if next_char != '' else '')
            #print(prefix,"start",start) 
            for end in arrangements(fifo[size+1:], others):
                #print(prefix,"add",start + end) 
                res.add(start + end)
        skip, fifo = fifo[0], fifo[1:]
        skipped += 1
        if len(fifo) < size:
            break
        if skip == "#":
            break
    return res
        

In [4]:
input_file = "input.txt"
file = open(input_file, 'r')
part1 = 0
for ix, line in enumerate(file):
    row, groups =line.split(' ')
    groups = tuple([int(g) for g in groups.split(',')])
    arr = len(arrangements(row,groups))
    part1 += arr
part1


6935

In [5]:
# That won't scale - DON'T RUN THIS
#input_file = "input.txt"
#file = open(input_file, 'r')
#part2 = 0
#for ix, line in enumerate(file):
#    row, groups =line.split(' ')
#    row = '?'.join([row]*5)
#    groups = groups.strip()
#    groups = ','.join([groups]*5)
#    groups = tuple([int(g) for g in groups.split(',')])
#    arr = len(arrangements(row,groups))
#    part2 += arr
#part2
# To slow

## Neat dynamic programming

In [6]:

@cache
def nb_arrangements( chunk, groups ):

    if len(groups) == 0: # work is done unless unmatched '#' left
        return 0 if '#' in chunk else 1
    if sum(groups) + len(groups) - 1 > len(chunk): # no room to end work
        return 0

    # FULLY RECURSIVE APPROACH
    if chunk[0] == '.':
        return nb_arrangements( chunk[1:], groups)

    cpt = 0
    if chunk[0] == '?': # considering the case it's a dammaged spring (like '.')
        cpt += nb_arrangements( chunk[1:], groups)

    group = groups[0]
    if '.' not in chunk[:group] and (len(chunk) == group or (len(chunk) > group and chunk[group] != '#')):
            # placing the first group of # is possible.
            # Let's consider it's done and count following arrangements
            cpt += nb_arrangements( chunk[group+1:], groups[1:])

    return cpt

In [7]:
part1 = 0
with open('input.txt', 'r') as f:
    for line in f.read().splitlines():
        row, groups = line.split(' ')
        groups = [int(x) for x in groups.split(',')]
        part1 += nb_arrangements( row , tuple(groups) ) # tuples pour pouvoir hasher...

part1

6935

In [8]:
part2 = 0
with open('input.txt', 'r') as f:
    for line in f.read().splitlines():
        chunk, groups = line.split(' ')
        chunk = '?'.join([chunk]*5)
        groups = ','.join([groups]*5)
        groups = [int(x) for x in groups.split(',')]
        part2 += nb_arrangements( chunk , tuple(groups) )

part2

3920437278260